# Movie Recommendations

Your company would like to start a movie studio and is asking us to recommend the best way foward. The main objective of this project is to produce three actionable recommendations for your company to take into consideration. With these recommendations your company can create their own successful movie studio.

# Goals

Here are the questions I would like to answer:
* What genres perform the best? 
* What is the ideal release date?
* Are there certain people involved in the most successful movies (directors, actors, or writers)?

## Success Metric

We will measure how much of the production budget was made back. A film is generally considered a success if it makes back at least 2x its budget. Therefore we will be judging success by ROI and looking for ROI of at least 100% to be considered a success.

ROI = ((worldwide gross - production budget) / production budget) x 100

## Data

* BOM Movie Gross (csv)
* IMDB (Database)
* Movie Budgets (csv)

## Methods

In [2]:
# import libraries
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
# connect database
conn = sqlite3.connect('../zippedData/im.db')

In [3]:
# review tables in the database (make sure bom is still added)
pd.read_sql("""

SELECT name
FROM sqlite_master
WHERE type = "table"

""", conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers
8,bom_movie_gross
9,rt_movie_info


### Clean Movie Basics Data

In [4]:
movie_basics=pd.read_sql("""SELECT * FROM movie_basics""",conn)

In [5]:
movie_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [11]:
# Add success metric to movie_basics
movies_summary=pd.read_sql("""
SELECT movie, production_budget, worldwide_gross, genres, runtime_minutes  
FROM movie_budgets budget
INNER JOIN movie_basics basic
ON TRIM(LOWER(budget.movie))=TRIM(LOWER(basic.primary_title))
""",conn)

In [12]:
movies_summary

,movie,production_budget,worldwide_gross,genres,runtime_minutes
0,Avatar,"$425,000,000","$2,776,345,279",Horror,93.0
1,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$1,045,663,875","Action,Adventure,Fantasy",136.0
2,Dark Phoenix,"$350,000,000","$149,762,350","Action,Adventure,Sci-Fi",113.0
3,Avengers: Age of Ultron,"$330,600,000","$1,403,013,963","Action,Adventure,Sci-Fi",141.0
4,Avengers: Infinity War,"$300,000,000","$2,048,134,200","Action,Adventure,Sci-Fi",149.0
...,...,...,...,...,...
3886,Cure,"$10,000","$94,596",None,NaN
3887,Bang,"$10,000",$527,None,NaN
3888,Newlyweds,"$9,000","$4,584","Comedy,Drama",95.0
3889,Red 11,"$7,000",$0,"Horror,Sci-Fi,Thriller",77.0


In [13]:
# Add to SQL data
movies_summary.to_sql('movies_summary',conn, if_exists='replace')

3891

In [14]:
movies_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3891 entries, 0 to 3890
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie              3891 non-null   object 
 1   production_budget  3891 non-null   object 
 2   worldwide_gross    3891 non-null   object 
 3   genres             3818 non-null   object 
 4   runtime_minutes    3397 non-null   float64
dtypes: float64(1), object(4)
memory usage: 152.1+ KB


### Add Success Metric to movies_summary table

In [15]:
movies_summary

,movie,production_budget,worldwide_gross,genres,runtime_minutes
0,Avatar,"$425,000,000","$2,776,345,279",Horror,93.0
1,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$1,045,663,875","Action,Adventure,Fantasy",136.0
2,Dark Phoenix,"$350,000,000","$149,762,350","Action,Adventure,Sci-Fi",113.0
3,Avengers: Age of Ultron,"$330,600,000","$1,403,013,963","Action,Adventure,Sci-Fi",141.0
4,Avengers: Infinity War,"$300,000,000","$2,048,134,200","Action,Adventure,Sci-Fi",149.0
...,...,...,...,...,...
3886,Cure,"$10,000","$94,596",None,NaN
3887,Bang,"$10,000",$527,None,NaN
3888,Newlyweds,"$9,000","$4,584","Comedy,Drama",95.0
3889,Red 11,"$7,000",$0,"Horror,Sci-Fi,Thriller",77.0


In [18]:
movies_summary['production_budget'] = movies_summary['production_budget'].replace({'\$': '', ',': ''}, regex=True).astype(float)
movies_summary['worldwide_gross'] = movies_summary['worldwide_gross'].replace({'\$': '', ',': ''}, regex=True).astype(float)

In [23]:
#Calculate net profit and add to DF
movies_summary['net_profit'] = movies_summary['worldwide_gross'] - movies_summary['production_budget']

In [24]:
#Calculate ROI and add to DF
movies_summary['roi'] = (movies_summary['net_profit'] / movies_summary['production_budget']) * 100

In [26]:
# Format net_profit with commas for thousands separator
movies_summary['net_profit'] = movies_summary['net_profit'].apply(lambda x: f"{x:,.2f}")

# Format ROI as percentage (with two decimal places)
movies_summary['roi'] = movies_summary['roi'].apply(lambda x: f"{x:,.2f}%")

In [28]:
# Format production_budget and worldwide_gross with commas for thousands separator
movies_summary['production_budget'] = movies_summary['production_budget'].apply(lambda x: f"${x:,.0f}")
movies_summary['worldwide_gross'] = movies_summary['worldwide_gross'].apply(lambda x: f"${x:,.0f}")

In [29]:
movies_summary

,movie,production_budget,worldwide_gross,genres,runtime_minutes,roi,net_profit
0,Avatar,"$425,000,000","$2,776,345,279",Horror,93.0,553.26%,"2,351,345,279.00"
1,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$1,045,663,875","Action,Adventure,Fantasy",136.0,154.67%,"635,063,875.00"
2,Dark Phoenix,"$350,000,000","$149,762,350","Action,Adventure,Sci-Fi",113.0,-57.21%,"-200,237,650.00"
3,Avengers: Age of Ultron,"$330,600,000","$1,403,013,963","Action,Adventure,Sci-Fi",141.0,324.38%,"1,072,413,963.00"
4,Avengers: Infinity War,"$300,000,000","$2,048,134,200","Action,Adventure,Sci-Fi",149.0,582.71%,"1,748,134,200.00"
...,...,...,...,...,...,...,...
3886,Cure,"$10,000","$94,596",None,NaN,845.96%,"84,596.00"
3887,Bang,"$10,000",$527,None,NaN,-94.73%,"-9,473.00"
3888,Newlyweds,"$9,000","$4,584","Comedy,Drama",95.0,-49.07%,"-4,416.00"
3889,Red 11,"$7,000",$0,"Horror,Sci-Fi,Thriller",77.0,-100.00%,"-7,000.00"


<bound method Series.info of count         3891
unique        2146
top       -100.00%
freq           377
Name: roi, dtype: object>

## What genres perform the best? 


## Ideal Run Time?

## Are there certain directors, writers or actors to involve in our productions?

## Results

### Business Recommendation 1

### Business Recommendation 2

### Business Recommendation 3